In [7]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score
# 1. Load the dataset
df=pd.read_csv('..\\data\\fraud.csv', index_col = 0)
# print(df.columns)
#Update the DataFrame df by keeping all rows and dropping the first column.
#Explanation:
# df.iloc is used to access rows and columns by position (not by labels).
# : means all rows.
# 1: means columns from position 1 to the end.
# Column index 0 is the first column, so this skips the first column.
df = df.iloc[:,1:]
# print(df.columns)
y = df['Class'].values

X = df.drop(columns = 'Class').values


In [9]:
# (rows, columns)
print(X.shape, y.shape)
# 2. checking the null value contain in the dataset or not
# df.info()
# print("--------------------------------")
# print(df.isnull().sum())
# print("--------------------------------")

# print(df.columns)
# print(df.isnull().sum())
print(df[['Class']].value_counts())
df['Class'].value_counts(normalize=True) * 100
#data is imbalance 98% not fraud, 2% fraud trans


(21693, 29) (21693,)
Class
0        21337
1          356
Name: count, dtype: int64


Class
0    98.358918
1     1.641082
Name: proportion, dtype: float64

In [11]:
# split the data
#60% training data and 40% testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                    test_size = 0.40, 
                                    random_state=1)

#Confusion Matrix Definition [[TN FP] [FN TP]]

True Negative (TN): Actual = 0, Predicted = 0 → Correctly predicted not fraud

False Positive (FP): Actual = 0, Predicted = 1 → Predicted fraud, but it’s not (false alarm)

False Negative (FN): Actual = 1, Predicted = 0 → Missed a fraud case! ❌

True Positive (TP): Actual = 1, Predicted = 1 → Correctly predicted fraud ✅

Why It Matters in Fraud Detection FP (False Positive): Inconvenience to a customer (e.g., transaction blocked)

FN (False Negative): Very dangerous — fraud not caught!

So in fraud detection:

Recall (how many actual frauds you caught) is very important.

You want to minimize FN, even at the cost of a few extra FPs.

             Predicted
            0       1
Actual 0 [TN FP] 1 [FN TP]

In [13]:
# ---- GAUSSIAN NB --------------------------------------------------
# works on continuous / numeric features (assumes Gaussian distribution)
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

y_pred_gnb = gnb.predict(X_test)

gnb_mat_clf = confusion_matrix(y_test, y_pred_gnb)
gnb_report_clf = classification_report(y_test, y_pred_gnb)
print("GNB Naive Bayes classification report")
print("Consufion Matrix: ",gnb_mat_clf.tolist())
print("Classification Report ")
print(gnb_report_clf)

# predict probabilities
# [:,1] means "probability of the positive class"
y_probs = gnb.predict_proba(X_test)[:, 1]

# calculate roc_auc
gnb_auc_score = roc_auc_score(y_test, y_probs)
print("GNB ROC-AUC:", gnb_auc_score)


GNB Naive Bayes classification report
Consufion Matrix:  [[8349, 192], [19, 118]]
Classification Report 
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      8541
           1       0.38      0.86      0.53       137

    accuracy                           0.98      8678
   macro avg       0.69      0.92      0.76      8678
weighted avg       0.99      0.98      0.98      8678

GNB ROC-AUC: 0.9712558658664049


In [19]:
# ---- MULTINOMIAL NB ----------------------------------------------
# works on count-based features (e.g. Bag-of-Words)
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha=1.0)
#Multinomial Naïve Bayes classifiers due to this fraud dataset values are negative and continuous date that not supported.
# mnb.fit(X_train, y_train)

# y_pred_mnb = mnb.predict(X_test)

# mnb_precision = precision_score(y_test, y_pred_mnb)
# mnb_recall = recall_score(y_test, y_pred_mnb)

# print("MultinomialNB -> Precision:", mnb_precision, ", Recall:", mnb_recall)

In [21]:
# ---- BERNOUILLI NB ------------------------------------------------
# works on binary (0/1) features only
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB(alpha=1.0)
bnb.fit(X_train, y_train)

y_pred_bnb = bnb.predict(X_test)

bnb_mat_clf = confusion_matrix(y_test, y_pred_bnb)
bnb_report_clf = classification_report(y_test, y_pred_bnb)
print("BNB Naive Bayes classification report")
print("Consufion Matrix: ",bnb_mat_clf.tolist())
print("Classification Report ")
print(bnb_report_clf)

# predict probabilities
# [:,1] means "probability of the positive class"
y_probs = bnb.predict_proba(X_test)[:, 1]

# calculate roc_auc
bnb_auc_score = roc_auc_score(y_test, y_probs)
print("BNB ROC-AUC:", bnb_auc_score)

# bnb_precision = precision_score(y_test, y_pred_bnb)
# bnb_recall = recall_score(y_test, y_pred_bnb)

# print("BernoulliNB  -> Precision:", bnb_precision, ", Recall:", bnb_recall)

BNB Naive Bayes classification report
Consufion Matrix:  [[8533, 8], [33, 104]]
Classification Report 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8541
           1       0.93      0.76      0.84       137

    accuracy                           1.00      8678
   macro avg       0.96      0.88      0.92      8678
weighted avg       1.00      1.00      1.00      8678

BNB ROC-AUC: 0.9551925149365406
